# Developing an Information Retrieval System with Document Ranking

This project aims to augment the Information Retrieval (IR) system developed in the previous assignments by incorporating different Document Ranking strategies. You should use the Cranfield collection as the dataset. You can find
that in the original format here or in the TREC XML format with binary tagging here.



## Project Overview

In this project, you will implement three different approaches for document ranking, including the vector space model,
the binary independence model, and the language model. Then, you need to compare these ranking models resorting
to the evaluation criteria in Lecture 7. Key components and functionalities are as follows:


### ⬜ Document Ranking – Space Vector Model

You will implement a function for document ranking utilizing
the tf-idf weighting approach. The function will take as input a query text and an integer indicating the number
of top documents to be retrieved. To this end, you might need to store additional information about the ”term
frequency” in each document and the ”document frequency” of each term. A good representation and sufficient
data storage facilitate you in implementing other models too. You can choose between different tf-idf variants
and need to discuss why your chosen approach is preferred.


#### tf-idf variants:

#### discussing why *** is perfect:

In [52]:
from pathlib import Path  # For standard paths that work on both windows and linux
import pickle  # For write/read dicts to/from files
import numpy as np
from nltk.tokenize import RegexpTokenizer  # For query tokenization
from nltk.stem import PorterStemmer  # For query stemming
import math # For calculating idf (log(N/df))
from collections import Counter # For calculating query's terms frequency

In [53]:
tokenizer = RegexpTokenizer(r'\w+')
ps = PorterStemmer()

In [54]:
# reading posting list from file
# posting list has been created in the last step (preprocessing)
posting_list = None
docs = None
tokenized_docs = None

with open(Path("files") / "posting_list.pkl", "rb") as f:
        posting_list = pickle.load(f)

with open(Path("files") / "docs.pkl", "rb") as f:
        docs = pickle.load(f)

with open(Path("files") / "tokenized_docs.pkl", "rb") as f:
        tokenized_docs = pickle.load(f)

In [55]:
print(f"but({len(posting_list['but'])} docs): [doc_id: frequency]",posting_list["but"])

but(219 docs): [doc_id: frequency] {13: 2, 24: 1, 27: 1, 43: 1, 48: 2, 61: 1, 65: 1, 71: 2, 93: 1, 109: 2, 113: 1, 115: 1, 124: 1, 125: 1, 131: 1, 137: 1, 139: 1, 148: 1, 151: 1, 152: 1, 155: 2, 159: 1, 167: 1, 172: 1, 175: 1, 178: 1, 184: 1, 187: 1, 188: 1, 190: 1, 192: 1, 198: 2, 200: 1, 201: 4, 203: 1, 205: 1, 208: 1, 209: 1, 211: 2, 213: 1, 220: 1, 226: 1, 228: 1, 240: 1, 243: 1, 246: 1, 251: 2, 254: 2, 260: 1, 265: 1, 271: 1, 282: 1, 283: 1, 291: 1, 295: 1, 328: 4, 337: 2, 346: 1, 347: 1, 351: 1, 369: 1, 374: 1, 389: 1, 403: 2, 416: 1, 430: 1, 440: 1, 451: 1, 458: 2, 476: 2, 483: 1, 489: 1, 498: 2, 510: 1, 514: 1, 518: 1, 519: 1, 520: 1, 521: 1, 526: 1, 535: 1, 544: 1, 546: 1, 555: 1, 561: 1, 563: 1, 565: 1, 566: 1, 568: 1, 569: 1, 571: 1, 588: 1, 594: 1, 599: 1, 603: 1, 635: 1, 639: 1, 642: 1, 651: 1, 659: 1, 660: 1, 662: 1, 666: 1, 670: 1, 672: 1, 678: 1, 684: 2, 685: 1, 691: 1, 703: 2, 717: 1, 720: 1, 726: 1, 730: 1, 738: 1, 747: 1, 751: 1, 752: 2, 755: 1, 756: 2, 759: 1, 762: 

In [56]:
# TODO: using sub-linear tf scaling or other approaches?

In [57]:
def cosine_score(query, k):
    query = [ps.stem(token) for token in tokenizer.tokenize(query)]

    query_term_frequency = Counter(query)

    docs_scores = np.zeros(len(docs))

    for term in posting_list:
        idf = math.log(
            len(docs) / len(posting_list[term]), 10
        )  # Log base is 10 in the slides

        term_query_weight = query_term_frequency[term] * idf

        for doc_id in posting_list[term]:
            tf = posting_list[term][doc_id]

            term_doc_weight = tf * idf

            docs_scores[doc_id] += term_query_weight * term_doc_weight

    for i in range(len(docs_scores)):
        docs_scores[i] /= len(tokenized_docs[i]) or 1

    return docs_scores.argsort()[-k:][::-1]

In [58]:
0.0011827143682338536/len(tokenized_docs[0])

8.508736462114055e-06

In [59]:
result = cosine_score(
    "what is the basic mechanism of the transonic aileron buzz .",
    5,
)
result

array([495, 642, 198, 878, 312], dtype=int64)

In [62]:
# Show the results
for i, doc_id in enumerate(result):
    print(f"================== Result {i + 1} ==================")
    print(docs[doc_id])
    print()

================== Result 1 ==================
a theory of transonic aileron buzz, neglecting viscous
effects .
  usaf-sponsored analysis of the unsteady perturbations of
two-dimensional transonic flow around an airfoil, where local supersonic
regions terminated by shock waves are present in the vicinity of the
airfoil .  viscous effects are neglected, and a linearized theory of
the perturbations due to harmonic oscillations of an aileron is
developed .  a series solution for the pressure distribution is obtained,
and numerical results for the nonsteady hinge moment, from the
first approximation to the solution, are presented .  as a result of
flutter analysis a stability boundary for transonic aileron buzz is
obtained .  comparison of the theoretical results with experimental
observations shows satisfactory agreement .

================== Result 2 ==================
an investigation of wing-aileron flutter using ground
launched rocket models .
  control surface flutter of the
wing tor